# Run DSI with PESTPP-IES

In [ ]:
%matplotlib inline
import os
import shutil
import pyemu
import helpers

Define the existing ies master dir

In [ ]:
ies_d = "master_ies_base"
assert os.path.exists(ies_d)

Load the control file from ies

In [ ]:
pst = pyemu.Pst(os.path.join(ies_d, "pest.pst"))

Get the prior observation (simulated results) ensemble - this will have outputs from the past and the future

In [ ]:
oe = pst.ies.obsen0

DSI works best when the histograms of the values of interest are (nearly) Guassian. So we can use a normal-score transform to help with this:

In [ ]:
transforms = [{"type": "normal_score"}]

Define the DSI instance

In [ ]:
dsi = pyemu.emulators.DSI(
    pst=pst,  # optional...
    data=oe,
    transforms=transforms,
    energy_threshold=0.9999,  # the truncated-svd energy threshold
    verbose=True,
)

Now we need to "fit" our statistical model, which for this form of DSI is pretty trivial

In [ ]:
dsi.fit()

Define the directory that will hold the dsi files

In [ ]:
dsi_d = "master_dsi"

In [ ]:
if os.path.exists(dsi_d):
    shutil.rmtree(dsi_d)

prepare the DSI input and outputs with a pest interface so that ies can use the dsi model

In [ ]:
dpst = dsi.prepare_pestpp(t_d=dsi_d, use_runstor=True)

In [ ]:
dpst

Define some options for ies to use with the dsi model

In [ ]:
dpst.pestpp_options["ies_num_reals"] = 300
dpst.pestpp_options["par_sigma_range"] = 6
dpst.control_data.noptmax = 3
dpst.pestpp_options.pop("parcov", None)
# use the same obs+noise ensemble we used before
dpst.pestpp_options["ies_obs_en"] = pst.pestpp_options["ies_obs_en"]
shutil.copy2(
    os.path.join(ies_d, pst.pestpp_options["ies_obs_en"]),
    os.path.join(dsi_d, pst.pestpp_options["ies_obs_en"]),
)
# use the same phi-factor file we used before
dpst.pestpp_options["ies_phi_factor_file"] = pst.pestpp_options["ies_phi_factor_file"]
shutil.copy2(
    os.path.join(ies_d, pst.pestpp_options["ies_phi_factor_file"]),
    os.path.join(dsi_d, pst.pestpp_options["ies_phi_factor_file"]),
)
# and also the same:
dpst.pestpp_options["ies_bad_phi_sigma"] = pst.pestpp_options["ies_bad_phi_sigma"]
dpst.pestpp_options

Save the control file

In [ ]:
dpst.write(os.path.join(dsi_d, "dsi.pst"))

DSI is fast, so fast, we dont need tcp/ip run mgmt, we can just use a simple external run mgr.  Start pestpp-ies in the dsi directory with the "/e" flag:

In [ ]:
pyemu.os_utils.run("pestpp-ies dsi.pst /e", cwd=dsi_d)

All this is the same as in the ies notebook:

In [ ]:
noptmax = None

In [ ]:
helpers.plot_ies_timeseries(dsi_d, pst_name="dsi.pst", noptmax=noptmax)

In [ ]:
helpers.plot_ies_forecasts(dsi_d, pst_name="dsi.pst", include_t=True, noptmax=noptmax)

In [ ]:
tags = [
    "npf-k-layer1",
    "npf-k-layer5",
    "npf-k33-layer3",
    "sto-ss-layer1",
    "sto-sy-layer1",
]

In [ ]:
for tag in tags:
    helpers.plot_ies_properties(dsi_d, tag, pst_name="dsi.pst", noptmax=noptmax)